In [ ]:
#workdir_ = '/glade/work/juliob/'
import sys
#######################################
# Leave this for now. But it should change to better
# method as here:
import os
This_module_path = os.getcwd()  #os.path.dirname(os.path.abspath(__file__))
workdir_ = os.path.join(This_module_path, '../../' )
# sys.path.append(utils_path)
# print( f" a path added in {__name__} {utils_path} ")

print( f" In {__name__} we have This_module_path={This_module_path} " )
print( f" In {__name__} we have workdir_={workdir_} " )
########################################
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
#sys.path.append(workdir_ + 'PyRegridding/Regridder/')
sys.path.append(workdir_ + 'PyRegridding/Utils/')
sys.path.append(workdir_ + 'myPythonTools/Plotting/')

# Own local packages
import AveragingUtils as Av
#import VertRegridFlexLL as Vrg  # This is toxic for some reason
import PlotUtil as Pu
import utils as uti
import validation_data as Val
import var_A_x_B as vAB
import MakePressures as MkP
import GridUtils as GrU
import LatLonMaps as LL

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( LL )
importlib.reload( uti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(vAB)
importlib.reload(MkP)


In [ ]:


A = uti.MakeDict4Exp( exp='fmthist_MM_x21_2' , user='juliob', subd='hist' , hsPat='cam.h0' , ymdPat='*' ,verbose=True, open_dataset=True )
B = uti.MakeDict4Exp( exp='fmthist_MM_x21_2' , user='juliob', subd='regridded' , hsPat='cam.h0' , ymdPat='*' ,verbose=True, open_dataset=True )

#X1 = xr.open_mfdataset( path_X ,data_vars='different', coords='different' )


In [ ]:
A.X['PRECT'] = A.X['PRECC'] + A.X['PRECL']
B.X['PRECT'] = B.X['PRECC'] + B.X['PRECL']


In [ ]:

fld='PRECT'
season='jja'
#UUc_s,years_c,months_c = Av.Seasonal( ds=Dc, season=season , fld='U',return_time=True)
aa_s,years_x,months_x = Av.Seasonal( ds=A.X , season=season , fld=fld ,return_time=True)
bb_s,years_x,months_x = Av.Seasonal( ds=B.X , season=season , fld=fld ,return_time=True)


In [ ]:
aa_s.shape

In [ ]:
cc_s,plat,plon = LL.ScripRegrid( fields=[  aa_s ] , Src='ne30pg3', Dst='fv1x1',RegridMethod='CONSERVE' )

In [ ]:
flds = ['p','v','o']


In [ ]:

clev=np.linspace( 0,20, num=21 )
scale=1000.*86400.
fig = plt.figure(figsize=(25, 8))
nx,ny=3,1

Axes = Pu.axes_def(n=1,nxplo=nx,nyplo=ny ) 
ax1 = fig.add_axes( Axes )
co1=ax1.tricontourf(A.X.lon,A.X.lat, scale*aa_s , levels=clev  )
cbar = plt.colorbar(co1, ax=ax1, shrink=.6)
ax1.set_title(f"{A.exp}/{A.subd} {fld} {season.upper()} {years_x[0]}-{years_x[-1]}" )

Axes = Pu.axes_def(n=2,nxplo=nx,nyplo=ny ) 
ax1 = fig.add_axes( Axes )
co1=ax1.contourf(B.X.lon,B.X.lat, scale*bb_s , levels=clev  )
cbar = plt.colorbar(co1, ax=ax1, shrink=.6)
ax1.set_title(f"{B.exp}/{B.subd} {fld} {season.upper()} {years_x[0]}-{years_x[-1]}" )

Axes = Pu.axes_def(n=3,nxplo=nx,nyplo=ny ) 
ax1 = fig.add_axes( Axes )
co1=ax1.contourf(plon, plat, scale*cc_s[0] , levels=clev  )
cbar = plt.colorbar(co1, ax=ax1, shrink=.6)
ax1.set_title(f"{A.exp} {fld} REGRIDDED HERE {season.upper()} {years_x[0]}-{years_x[-1]}" )



In [ ]:

fields=[ scale*bb_s , scale*cc_s[0]  ]
lons=[B.X.lon,plon]
lats=[B.X.lat,plat,plat,plat] 
cmaps = ['terrain_r','terrain_r']
clevs=[clev,clev]

LL.Maps( fields=fields ,lons=lons ,lats=lats ,clevs=clevs,cmaps=cmaps )